# Librerias

In [11]:
# Lectura de archivos
import os
import xlrd
import xlwt
import openpyxl
import xlsxwriter
from openpyxl import Workbook
from openpyxl import load_workbook

# Manejo de matrices y preprocesamiento
import numpy as np
import pandas as pd

# Graficos
from matplotlib import pyplot as plt

# Red Neuronal
import keras.backend as K
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import EarlyStopping

# Ocultar Warnings
import warnings
warnings.filterwarnings("ignore")

## El siguiente cuadro solo correr una vez

In [17]:
#columns=("iter", "numBeta","diasPredecir", "n_features","n_steps", "model", "activation","error_5d_2", "error_10d_2", "error_5d_abs", "error_10d_abs")
results = pd.DataFrame()
preds = {}
num=0
# xls
def createBook():
    libro = Workbook()
    #sheet = libro.add_sheet("vacio")
    libro.save("predicciones.xlsx")  
createBook()

# Funciones

In [12]:
np.random.seed(7)

# cargar el archivo
def openFile(file, splitPoint, Beta):
    dataset = pd.read_excel(file, header=(0), sheet_name=str("Nominales_B"+Beta+"_Y"))
    dataset = dataset.iloc[splitPoint[0]:splitPoint[1]]
    dataset = dataset.values
    dataset = dataset.astype('float32')

    sequence=list()
    for i in range(0,len(dataset)):
        sequence.append(dataset[i][0])

    dataset = np.array(sequence)
    return dataset

# partir la secuencia univariada en samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

## modelos LSTM

# Vanilla LSTM
def vanillaLSTM(activation, n_steps, n_features):
    model = Sequential()
    model.add(LSTM(50, activation=activation, input_shape=(n_steps, n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# StackedLSTM
def stackedLSTM(activation, n_steps, n_features):
    model = Sequential()
    model.add(LSTM(50, activation=activation, return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(50, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Bidirectional
def bidirectionalLSTM(activation, n_steps, n_features):
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation=activation), input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

#MSE & AEM
def medidaError(param_model, param_pred):
    
    error_5d_2 = 0
    error_10d_2 = 0
    for i in range(0, param_model["iter"]):
        error_i_5d = (param_pred["yhat_5d"][i]-param_pred["y_5d"][i])**2
        error_5d_2= error_5d_2+error_i_5d
        error_i_10d = (param_pred["yhat_10d"][i]-param_pred["y_10d"][i])**2
        error_10d_2= error_10d_2+error_i_10d
    
    error_5d_abs = 0
    error_10d_abs = 0
    for i in range(0, param_model["iter"]):
        error_i_5d = abs(param_pred["yhat_5d"][i]-param_pred["y_5d"][i])
        error_5d_abs= error_5d_abs+error_i_5d
        error_i_10d = abs(param_pred["yhat_10d"][i]-param_pred["y_10d"][i])
        error_10d_abs= error_10d_abs+error_i_10d
    
    return error_5d_2, error_10d_2, error_5d_abs, error_10d_abs

# LSTM

In [23]:
def testLSTM(iteracion, numBeta, diasPredecir, n_features, n_steps, model, activation, Beta, resultados, num):
    """
    iteracion: observaciones en el tiempo en las cuales voy a hacer la prediccion
    numBeta: cantidad de betas que traigo del B_# original
    diasPredecir: numero de dias a predecir
    n_features: numero de variables X # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_steps: numero de betas empleados como train
    
    
    """
    dataset = openFile('00_DatosOriginalesLag.xlsx', (3230, 4010), Beta)
    # matriz para ir guardando los resultados
    matrizMagica = np.zeros((500,500))

    yhat_5d = []
    y_5d = []
    yhat_10d = []
    y_10d = []
    
    model = model(activation, n_steps, n_features)

    for i in range(0,iteracion): # -10 porque si no fallara cuando busque i+10

        matrizMagica[0:i+numBeta,i] = dataset[0:i+numBeta]

        for j in range(0,diasPredecir):
            # define input sequence
            raw_seq  = list(matrizMagica[0:i+numBeta+j,i])
            # split into samples
            X, y = split_sequence(raw_seq, n_steps)
            # reshape from [samples, timesteps] into [samples, timesteps, features]
            X = X.reshape((X.shape[0], X.shape[1], n_features))
            # fit model
            early_stop = EarlyStopping(monitor='loss', patience=2, verbose=0)
            model.fit(X, y, epochs=200, verbose=0, batch_size=3, callbacks=[early_stop])
            # demonstrate prediction
            x_input = np.array(raw_seq[-n_steps:])
            x_input = x_input.reshape((1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
            # guardar el dato
            matrizMagica[i+numBeta+j,i] = yhat[0][0]
            # guardar dia 5 y dia 10
            if j == 4:
                yhat_5d.append(yhat[0][0])
                y_5d.append(dataset[i+numBeta+j])
                #print("i: {} j: {} Prediccion: {:f} Original: {:f}".format(i, j, yhat[0][0], dataset[i+numBeta+j]))
            elif j == 9:
                yhat_10d.append(yhat[0][0])
                y_10d.append(dataset[i+numBeta+j])
                print("i: {} j: {} Prediccion: {:f} Original: {:f}".format(i, j, yhat[0][0], dataset[i+numBeta+j]))
            else:
                pass
            
    
    
    param_model = {"iter":iteracion, "numBeta": numBeta, 
             "diasPredecir": diasPredecir, "n_features": n_features, 
             "n_steps": n_steps, "model": model, "activation": activation}
            #print("i: {} j: {} prediccion: {:f}".format(i, j, yhat[0][0]))
    param_pred = {"yhat_5d": yhat_5d, "y_5d": y_5d, "yhat_10d": yhat_10d, "y_10d": y_10d}
    
    #calcula errores
    error_5d_2, error_10d_2, error_5d_abs, error_10d_abs = medidaError(param_model, param_pred)
    
    print("El error cuadrado medio para 5 días es : ",error_5d_2)
    print("El error cuadrado medio para 10 días es : ", error_10d_2)
    print( "El error absoluto medio para 5 días es :  ",error_5d_abs)
    print("El error absoluto medio para 10 días es : ", error_10d_abs)    
    
    results_iter = pd.DataFrame([iteracion, numBeta, diasPredecir, n_features, n_steps, "VanillaLSTM", activation, error_5d_2, error_10d_2, error_5d_abs, error_10d_abs])
    predicciones = pd.DataFrame(np.array([yhat_5d, y_5d,yhat_10d, y_10d]))
    preds["resultado_"+str(num)] = predicciones.T
    resultados = pd.concat([resultados, results_iter], axis=1)
    
    num=num+1
    
    return model, resultados, num

In [24]:
#iteracion, numBeta, n_features, n_steps, model, activation
model_1, results_1, num = testLSTM(iteracion=30, numBeta=30, diasPredecir=10, n_features=1, n_steps=5, model=bidirectionalLSTM, activation="tanh", Beta="1", resultados=results, num=num)

i: 0 j: 9 Prediccion: -0.026006 Original: -0.026503
i: 1 j: 9 Prediccion: -0.027675 Original: -0.028566
i: 2 j: 9 Prediccion: -0.029149 Original: -0.027819
El error cuadrado medio para 5 días es :  1.642646419639654e-05
El error cuadrado medio para 10 días es :  2.8113260117125505e-06
El error absoluto medio para 5 días es :   0.006227917969226837
El error absoluto medio para 10 días es :  0.0027187634259462357


In [25]:
#iteracion, numBeta, n_features, n_steps, model, activation
model_2, results_2, num = testLSTM(iteracion=30, numBeta=30, diasPredecir=10, n_features=1, n_steps=10, model=bidirectionalLSTM, activation="tanh", Beta="1", resultados=results_1, num=num)

i: 0 j: 9 Prediccion: -0.024533 Original: -0.026503
i: 1 j: 9 Prediccion: -0.026475 Original: -0.028566
i: 2 j: 9 Prediccion: -0.026674 Original: -0.027819
El error cuadrado medio para 5 días es :  7.017506095559345e-06
El error cuadrado medio para 10 días es :  9.561851252021941e-06
El error absoluto medio para 5 días es :   0.004138831049203873
El error absoluto medio para 10 días es :  0.005205454304814339


In [27]:
#iteracion, numBeta, n_features, n_steps, model, activation
model_3, results_3, num = testLSTM(iteracion=30, numBeta=30, diasPredecir=10, n_features=1, n_steps=15, model=bidirectionalLSTM, activation="tanh", Beta="1", resultados=results_2, num=num)

i: 0 j: 9 Prediccion: -0.024066 Original: -0.026503
i: 1 j: 9 Prediccion: -0.026920 Original: -0.028566
i: 2 j: 9 Prediccion: -0.025095 Original: -0.027819
El error cuadrado medio para 5 días es :  5.761845929225934e-06
El error cuadrado medio para 10 días es :  1.6066247681276874e-05
El error absoluto medio para 5 días es :   0.0033069979399442673
El error absoluto medio para 10 días es :  0.006806744262576103


In [28]:
#iteracion, numBeta, n_features, n_steps, model, activation
model_4, results_4, num  = testLSTM(iteracion=30, numBeta=30, diasPredecir=10, n_features=1, n_steps=20, model=bidirectionalLSTM, activation="tanh", Beta="1", resultados=results_3, num=num)

i: 0 j: 9 Prediccion: -0.025734 Original: -0.026503
i: 1 j: 9 Prediccion: -0.024447 Original: -0.028566
i: 2 j: 9 Prediccion: -0.024401 Original: -0.027819
El error cuadrado medio para 5 días es :  3.1511924837406002e-06
El error cuadrado medio para 10 días es :  2.9248770866647372e-05
El error absoluto medio para 5 días es :   0.002906324341893196
El error absoluto medio para 10 días es :  0.008306657895445824


In [29]:
#iteracion, numBeta, n_features, n_steps, model, activation
model_5, results_5, num  = testLSTM(iteracion=30, numBeta=30, diasPredecir=10, n_features=1, n_steps=25, model=bidirectionalLSTM, activation="tanh", Beta="1", resultados=results_4, num=num)

i: 0 j: 9 Prediccion: -0.024645 Original: -0.026503
i: 1 j: 9 Prediccion: -0.023001 Original: -0.028566
i: 2 j: 9 Prediccion: -0.024217 Original: -0.027819
El error cuadrado medio para 5 días es :  2.4958966155334827e-06
El error cuadrado medio para 10 días es :  4.7394667338388624e-05
El error absoluto medio para 5 días es :   0.0026401337236166
El error absoluto medio para 10 días es :  0.011024346575140953


In [31]:
results = results_5.T
results.columns = ["iter", "numBeta","diasPredecir", "n_features","n_steps", "model", "activation","error_5d_2", "error_10d_2", "error_5d_abs", "error_10d_abs"]
results

,iter,numBeta,diasPredecir,n_features,n_steps,model,activation,error_5d_2,error_10d_2,error_5d_abs,error_10d_abs
0,3,30,10,1,5,VanillaLSTM,tanh,7.24388e-06,3.6101e-06,0.00330433,0.00308211
0,3,30,10,1,10,VanillaLSTM,tanh,2.79298e-05,1.68069e-05,0.00773725,0.00565595
0,3,30,10,1,15,VanillaLSTM,tanh,3.93098e-06,2.36269e-05,0.00342949,0.00740244
0,3,30,10,1,5,VanillaLSTM,tanh,1.64265e-05,2.81133e-06,0.00622792,0.00271876
0,3,30,10,1,10,VanillaLSTM,tanh,7.01751e-06,9.56185e-06,0.00413883,0.00520545
0,3,30,10,1,15,VanillaLSTM,tanh,5.76185e-06,1.60662e-05,0.003307,0.00680674
0,3,30,10,1,20,VanillaLSTM,tanh,3.15119e-06,2.92488e-05,0.00290632,0.00830666
0,3,30,10,1,25,VanillaLSTM,tanh,2.4959e-06,4.73947e-05,0.00264013,0.0110243


In [32]:
writer = pd.ExcelWriter("predicciones.xlsx", engine="xlsxwriter")
for sheet_name in preds.keys():
    preds[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)
writer.save()

In [33]:
results.to_excel(r"./resultados.xlsx")